In [ ]:
import numpy as np
import pandas as pd

## **1. Data Definition**

### Customers Data

In [ ]:
demands = [0, 5, 20, 10, 20, 85, 65, 30, 20, 70, 30]
customer_data = pd.DataFrame({'customer': range(1, 12), 'd': demands})
customer_data

### Vehicles Data

In [ ]:
num_vehicles = 6
vehicle_capacity = 100

### Routes Costs

In [ ]:
distance_matrix = np.array([
    [0, 13, 6, 55, 93, 164, 166, 168, 169, 231, 212],
    [13, 0, 11, 66, 261, 175, 177, 179, 180, 239, 208],
    [6, 11, 0, 60, 97, 168, 171, 173, 174, 239, 209],
    [55, 66, 60, 0, 82, 113, 115, 117, 117, 295, 265],
    [93, 261, 97, 82, 0, 113, 115, 117, 118, 333, 302],
    [164, 175, 168, 113, 113, 0, 6, 7, 2, 403, 374],
    [166, 177, 171, 115, 115, 6, 0, 8, 7, 406, 376],
    [168, 179, 173, 117, 117, 7, 8, 0, 3, 408, 378],
    [169, 180, 174, 117, 118, 2, 7, 3, 0, 409, 379],
    [231, 239, 239, 295, 333, 403, 406, 408, 409, 0, 46],
    [212, 208, 209, 265, 302, 374, 376, 378, 379, 46, 0]
])

distances = pd.DataFrame(distance_matrix, index=range(1, 12), columns=range(1, 12))
distances

## **2. Graph**

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

# Add nodes (customers) to the graph
for i in range(1, 12):
    G.add_node(i)

# G.add_node(0)
# G.add_node(12)

# Add edges (connections between customers) to the graph
for i in range(1, 12):
    for j in range(i + 1, 12):
        if distance_matrix[i - 1, j - 1] != 0:
            G.add_edge(i, j, weight=distance_matrix[i - 1, j - 1])

# Draw the graph
pos = nx.spring_layout(G)  # Define node positions
nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', font_size=10, font_weight='bold')  # Draw nodes
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_size=5)  # Draw edge labels
plt.title('Customer Graph with Distances')
plt.show()

## **3. Genetic Algorithm**

In [ ]:
class GeneticAlgorithm:
    def __init__(self, num_customers, num_vehicles, capacity, distance_matrix, demands, population_size=100, num_generations=100, crossover_rate=0.8, mutation_rate=0.1):
        self.num_customers = num_customers
        self.num_vehicles = num_vehicles
        self.capacity = capacity
        self.distance_matrix = distance_matrix
        self.demands = demands
        self.population_size = population_size
        self.num_generations = num_generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.population = []

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = []
            for _ in range(self.num_vehicles):
                route = np.random.permutation(range(1, self.num_customers + 1))
                individual.append(route)
            self.population.append(individual)

    def evaluate_route(self, route):
        total_distance = 0
        demand = 0
        current_node = 0  # Start from depot
        for customer in route:
            next_node = customer
            total_distance += self.distance_matrix[current_node - 1, next_node - 1]  # Indexing starts from 0
            demand += self.demands[next_node - 1]  # Indexing starts from 0
            if demand > self.capacity:
                return float('inf')  # Penalize routes exceeding capacity
            current_node = next_node
        total_distance += self.distance_matrix[current_node - 1, 0]  # Return to depot
        return total_distance

    def evaluate_individual(self, individual):
        total_distance = 0
        for route in individual:
            total_distance += self.evaluate_route(route)
        return total_distance

    def select_parents(self):
        # Tournament selection
        parents = []
        for _ in range(self.population_size):
            tournament_indices = np.random.choice(self.population_size, size=2, replace=False)
            parent1 = min(tournament_indices, key=lambda x: self.evaluate_individual(self.population[x]))
            tournament_indices = np.delete(tournament_indices, np.where(tournament_indices == parent1))
            parent2 = min(tournament_indices, key=lambda x: self.evaluate_individual(self.population[x]))
            parents.append((parent1, parent2))
        return parents

    def crossover(self, parent1, parent2):
        # Convert parent1 and parent2 to lists
        parent1 = list(parent1)
        parent2 = list(parent2)
        
        # Ordered crossover
        crossover_point = np.random.randint(1, self.num_customers)  # Choose crossover point
        
        # Convert the selected genes to tuples for faster membership testing
        set_parent1 = tuple(parent1[:crossover_point])
        set_parent2 = tuple(parent2[:crossover_point])
        
        # Create child1 and child2
        child1 = parent1[:crossover_point] + [gene for gene in parent2 if gene.tolist() not in [tuple(route) for route in set_parent1]]
        child2 = parent2[:crossover_point] + [gene for gene in parent1 if gene.tolist() not in [tuple(route) for route in set_parent2]]

        return child1, child2

    def mutate(self, individual):
        # Swap mutation
        if np.random.rand() < self.mutation_rate:
            mutate_indices = np.random.choice(len(individual), size=2, replace=False)
            individual[mutate_indices[0]], individual[mutate_indices[1]] = individual[mutate_indices[1]], individual[mutate_indices[0]]

    def evolve_population(self):
        new_population = []
        parents = self.select_parents()
        for parent1, parent2 in parents:
            child1, child2 = self.crossover(self.population[parent1], self.population[parent2])
            self.mutate(child1)
            self.mutate(child2)
            new_population.extend([child1, child2])
        self.population = new_population[:self.population_size]

    def solve(self):
        self.initialize_population()
        best_cost = float('inf')
        best_routes = None
        for generation in range(self.num_generations):
            self.evolve_population()
            current_best_individual = min(self.population, key=self.evaluate_individual)
            current_best_cost = self.evaluate_individual(current_best_individual)
            print(f"Generation {generation+1}, Best Cost: {current_best_cost}")
            if current_best_cost < best_cost:
                best_cost = current_best_cost
                best_routes = current_best_individual
        return best_cost, best_routes


## **4. Genetic Algorithm Execution**

In [ ]:
# Solve the problem
genetic_alg = GeneticAlgorithm(
    num_customers=len(demands) - 1,
    num_vehicles=num_vehicles,
    capacity=vehicle_capacity,
    distance_matrix=distance_matrix,
    demands=demands
)

best_cost, best_routes = genetic_alg.solve()

if best_routes is not None:
    print("Best Cost:", best_cost)
    print("Best Routes:")
    for i, route in enumerate(best_routes):
        print("Vehicle", i+1, ":", route)
else:
    print("No feasible solution found within the specified number of generations.")


## **5. Optimal Route Visualization**

## **6. Results Analysis**